In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from collections import Counter
import math

- [x] Capturar Número de Vagas
- [x] Criar lista de links
- [x] Criar lista com descrições vagas
- [x] transformar em um DataFrame
- [x] Calcular tf_idf de palavras por vaga
- [ ] Recomendar 10 melhores vagas a partir de palavras-chave inseridas pelo usuario

# Get number of jobs

In [2]:
paginator = 0
r = requests.get(f'https://br.indeed.com/empregos?q=cientista+de+dados&l=brasil&start={paginator}')
page = BeautifulSoup(r.content, 'html.parser')

In [3]:
n_jobs = int(page.find(id='searchCountPages').text.split()[3])
n_jobs

250

# Get opportunity links

In [4]:
job_links = []
for i in range(0, n_jobs, 10):
    r = requests.get(f'https://br.indeed.com/empregos?q=cientista+de+dados&l=brasil&start={i}')
    page = BeautifulSoup(r.content, 'html.parser')
    anchors = page.find_all('a', class_='jobtitle')
    for a in anchors:
        job_links.append('https://www.indeed.com' + a.get('href'))

# Get list of job description

In [5]:
job_description = []
bad_attemps = []
for link in job_links:
    r = requests.get(link)
    page = BeautifulSoup(r.content, 'html.parser')
    try:
        description = page.find('div', class_='jobsearch-jobDescriptionText').text.replace('\n', ' ')
    except:
        bad_attemps.append(link)
        continue
    job_description.append(description)
        
for link in bad_attemps:
    job_links.remove(link)

In [6]:
print(len(job_links))
print(len(job_description))

363
363


In [7]:
jobs = pd.DataFrame({'URL': job_links, 'description': job_description})

In [10]:
symbols = "!\"#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"

def remove_symbols(description):
    for i in symbols:
        description = description.replace(i, ' ')
    return description

def remove_singles(description):
    new_text = ""
    for word in description.split(' '):
        if len(word) > 1:
            new_text += " " + word
    return new_text

# transform to lowercase
jobs['description'] = jobs['description'].apply(lambda x: x.lower())

# removing the symbols
jobs['description'] = jobs['description'].apply(lambda x: remove_symbols(x))

# removing single characters
jobs['description'] = jobs['description'].apply(lambda x: remove_singles(x))
jobs

,URL,description
0,https://www.indeed.com/company/Postmetria/jobs...,atribuições atuar em demandas relacionadas pr...
1,https://www.indeed.com/rc/clk?jk=dd1788e91fc88...,profissional formado em exatas com interesse ...
2,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...
3,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...
4,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...
...,...,...
358,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...
359,https://www.indeed.com/rc/clk?jk=85a3575bc36c0...,descrição da empresa descomplica uma platafor...
360,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...
361,https://www.indeed.com/rc/clk?jk=f94e4bea2baef...,atividades que você irá fazer produzir soluçõ...


# Tf_idf calc

### Set of words in corpus

In [12]:
words = []
for description in jobs['description']:
    for word in description.split(' '):
        words.append(word.strip(symbols))
word_set = set(words)
word_set.remove('')

### word counting

In [13]:
def word_count(description):
    word_dict = {}
    for i in word_set:
        word_dict[i] = description.count(i)
    return word_dict

In [14]:
jobs['word_count'] = jobs['description'].apply(word_count)
jobs

,URL,description,word_count
0,https://www.indeed.com/company/Postmetria/jobs...,atribuições atuar em demandas relacionadas pr...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
1,https://www.indeed.com/rc/clk?jk=dd1788e91fc88...,profissional formado em exatas com interesse ...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
2,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
3,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
4,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
...,...,...,...
358,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
359,https://www.indeed.com/rc/clk?jk=85a3575bc36c0...,descrição da empresa descomplica uma platafor...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."
360,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'correta': 0, 'número': 0, 'hard': 1, 'marech..."
361,https://www.indeed.com/rc/clk?jk=f94e4bea2baef...,atividades que você irá fazer produzir soluçõ...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech..."


### Term frequencies in job description

In [15]:
def tf(counter):
    tf_dict = {}
    counter_len = sum(counter.values())
    for key, value in counter.items():
        tf_dict[key] = value / counter_len
    return tf_dict

In [16]:
jobs['tf'] = jobs['word_count'].apply(tf)
jobs

,URL,description,word_count,tf
0,https://www.indeed.com/company/Postmetria/jobs...,atribuições atuar em demandas relacionadas pr...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
1,https://www.indeed.com/rc/clk?jk=dd1788e91fc88...,profissional formado em exatas com interesse ...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
2,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
3,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
4,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
...,...,...,...,...
358,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
359,https://www.indeed.com/rc/clk?jk=85a3575bc36c0...,descrição da empresa descomplica uma platafor...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
360,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'correta': 0, 'número': 0, 'hard': 1, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0003..."
361,https://www.indeed.com/rc/clk?jk=f94e4bea2baef...,atividades que você irá fazer produzir soluçõ...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."


### Inverse Document Frequencies

In [17]:
doc_freq = {}
for word in word_set:
    doc_freq[word] = 0

for word in word_set:
    for word_count in jobs['word_count']:
        if word_count[word] != 0:
            doc_freq[word] += 1

In [18]:
idf = {}
for word, freq in doc_freq.items():
    idf[word] = math.log(len(jobs)/(1 + freq))

### TF-IDF calc

In [19]:
def tfidf_calc(tf):
    tfidf = {}
    for word in tf.keys():
        tfidf[word] = tf[word] * idf[word]
    return tfidf

In [20]:
jobs['tfidf'] = jobs['tf'].apply(tfidf_calc)
jobs

,URL,description,word_count,tf,tfidf
0,https://www.indeed.com/company/Postmetria/jobs...,atribuições atuar em demandas relacionadas pr...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
1,https://www.indeed.com/rc/clk?jk=dd1788e91fc88...,profissional formado em exatas com interesse ...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
2,https://www.indeed.com/rc/clk?jk=1ef88d6418dca...,conheça nossa banda inovadora diferente diver...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
3,https://www.indeed.com/rc/clk?jk=6da90d3e35dc3...,nossa gente aqui na cervejaria ambev sonha gr...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
4,https://www.indeed.com/rc/clk?jk=7689940b1824c...,vaga de um cliente da talentbrand descrição d...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
...,...,...,...,...,...
358,https://www.indeed.com/rc/clk?jk=cc0f3f8cafc80...,eleflow nasceu para tornar big data analytics...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
359,https://www.indeed.com/rc/clk?jk=85a3575bc36c0...,descrição da empresa descomplica uma platafor...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."
360,https://www.indeed.com/rc/clk?jk=061a8dd5c5308...,descomplica uma plataforma de educação digita...,"{'correta': 0, 'número': 0, 'hard': 1, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0003...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0012..."
361,https://www.indeed.com/rc/clk?jk=f94e4bea2baef...,atividades que você irá fazer produzir soluçõ...,"{'correta': 0, 'número': 0, 'hard': 0, 'marech...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '...","{'correta': 0.0, 'número': 0.0, 'hard': 0.0, '..."


# Most important words of all jobs

In [21]:
word_importance = {}

In [22]:
for word in word_set:
    word_importance[word] = 0

In [23]:
for tfidf in jobs['tfidf']:
    for word, importance in word_importance.items():
        word_importance[word] += tfidf[word]

In [24]:
df_importance = pd.DataFrame({'word': word_importance.keys(), 'importance': word_importance.values()})
df_importance.sort_values(['importance'], ascending=False).head(10)

,word,importance
1274,learning,0.329360
6648,learn,0.292820
4255,machine,0.290123
4450,vale,0.288029
5420,modelo,0.282490
189,you,0.275352
4816,marketing,0.275000
2772,market,0.270557
3308,modelos,0.257906
3225,mode,0.253661


# Retrieving 10 bests job for user enter

In [74]:
user_enter = 'Conhecimentos em programação, python, turicreate, Sklearn, pandas, numpy,\
analytics, machine learning, criação de modelos preditivos.\
Conhecimentos básicos em sql, e bancos de dados relacionais e não relacionais.'
user_enter_filtered = remove_symbols(user_enter.lower())
user_enter_words = []
for i in user_enter_filtered.split(' '):
    if i != '':
        user_enter_words.append(i)
user_enter_words

['conhecimentos',
 'em',
 'programação',
 'python',
 'turicreate',
 'sklearn',
 'pandas',
 'numpy',
 'analytics',
 'machine',
 'learning',
 'criação',
 'de',
 'modelos',
 'preditivos',
 'conhecimentos',
 'básicos',
 'em',
 'sql',
 'e',
 'bancos',
 'de',
 'dados',
 'relacionais',
 'e',
 'não',
 'relacionais']

### Exctract user enter vector

In [75]:
user_enter_count = {}
for word in jobs['tfidf'][0].keys():
    user_enter_count[word] = 0

In [76]:
for word in user_enter_words:
    try:
        user_enter_count[word] += 1
    except:
        continue
user_enter_count['machine']

1

In [77]:
def cosine_similarity(tfidf):
    tfidf_vec = np.array(list(tfidf.values()))
    user_vec = np.array(list(user_enter_count.values()))
    sim = (tfidf_vec.dot(user_vec))/np.linalg.norm(tfidf_vec)*np.linalg.norm(user_vec)
    return sim

In [78]:
jobs['cos_sim'] = jobs['tfidf'].apply(cosine_similarity)

In [79]:
response = jobs.sort_values('cos_sim', ascending=False)[:10]

In [80]:
for i in response['URL']:
    print(i)

https://www.indeed.com/rc/clk?jk=64f439f63486103e&fccid=114efd88673ee90c&vjs=3
https://www.indeed.com/rc/clk?jk=71c0f9b52237a117&fccid=f278fd54bec2ca13&vjs=3
https://www.indeed.com/rc/clk?jk=2a04ebcc77ab5d65&fccid=b1def06b42bdc2ee&vjs=3
https://www.indeed.com/rc/clk?jk=1fb984ea3bddae0b&fccid=e5ea1ff710ce39ca&vjs=3
https://www.indeed.com/rc/clk?jk=47f4fad745bf9b9d&fccid=c010d2c764def3f0&vjs=3
https://www.indeed.com/rc/clk?jk=4ceffbe09706b379&fccid=c010d2c764def3f0&vjs=3
https://www.indeed.com/rc/clk?jk=0b4d88427aadc527&fccid=1abbffd7f44ec1e9&vjs=3
https://www.indeed.com/rc/clk?jk=c6fb76f38201c168&fccid=4bbf7251d41a576b&vjs=3
https://www.indeed.com/rc/clk?jk=3a722edb8332739a&fccid=22d7ce555383096d&vjs=3
https://www.indeed.com/rc/clk?jk=3a722edb8332739a&fccid=22d7ce555383096d&vjs=3
